<a href="https://colab.research.google.com/github/RohanKatyayani/finrisk-copilot/blob/main/llama_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing dependencies for LLaMA + LoRA
!pip install -q transformers accelerate bitsandbytes peft datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.4 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="data/german_credit_explanations.jsonl")

# Preview
print(dataset)
print(dataset["train"][0])

FileNotFoundError: Unable to find '/content/data/german_credit_explanations.jsonl'

In [3]:
from google.colab import files
uploaded = files.upload()

Saving german_credit_explanations.jsonl to german_credit_explanations.jsonl


In [4]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="german_credit_explanations.jsonl")

# Preview
print(dataset)
print(dataset["train"][0])

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'label', 'explanation'],
        num_rows: 1000
    })
})
{'input': 'status=... < 100 DM, duration=6, savings=unknown/no savings account, employment=... >= 7 years, amount=1169, age=67', 'label': 'bad', 'explanation': 'Application denied due to stable financial profile.'}


In [5]:
from datasets import load_dataset

# Loading the uploaded dataset
dataset = load_dataset("json", data_files="german_credit_explanations.jsonl")

# Preview
print(dataset)
print(dataset['train'][0])  # Show first sample

DatasetDict({
    train: Dataset({
        features: ['input', 'label', 'explanation'],
        num_rows: 1000
    })
})
{'input': 'status=... < 100 DM, duration=6, savings=unknown/no savings account, employment=... >= 7 years, amount=1169, age=67', 'label': 'bad', 'explanation': 'Application denied due to stable financial profile.'}


Example format for training

### Instruction:
You are a bank risk officer. Given customer credit information, decide if the application is good or bad and explain why.

### Input:
status=... < 100 DM, duration=6, savings=unknown/no savings account, employment=... >= 7 years, amount=1169, age=67

### Response:
Label: bad
Explanation: Application denied due to stable financial profile.

In [6]:
def format_example(example):
    return {
        "text": f"""### Instruction:
You are a bank risk officer. Given customer credit information, decide if the application is good or bad and explain why.

### Input:
{example['input']}

### Response:
Label: {example['label']}
Explanation: {example['explanation']}"""
    }

# Apply formatting
dataset = dataset.map(format_example)

# Check one example
print(dataset['train'][0]['text'])

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

### Instruction:
You are a bank risk officer. Given customer credit information, decide if the application is good or bad and explain why.

### Input:
status=... < 100 DM, duration=6, savings=unknown/no savings account, employment=... >= 7 years, amount=1169, age=67

### Response:
Label: bad
Explanation: Application denied due to stable financial profile.


In [2]:
!git clone https://github.com/RohanKatyayani/finrisk-copilot.git
%cd finrisk-copilot

Cloning into 'finrisk-copilot'...
remote: Enumerating objects: 92, done.
remote: Counting objects: 100% (92/92), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 92 (delta 31), reused 70 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (92/92), 280.48 KiB | 1.91 MiB/s, done.
Resolving deltas: 100% (31/31), done.
/content/finrisk-copilot


In [3]:
!pip install -q datasets transformers accelerate

In [4]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="data/credit_instructions.jsonl")

print(dataset)
print(dataset["train"][0])  # show first sample

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 1000
    })
})
{'instruction': 'Explain the credit risk decision for the following applicant profile.', 'input': 'status=... < 100 DM, duration=6, savings=unknown/no savings account, employment=... >= 7 years, amount=1169, age=67', 'output': 'Decision: bad. Explanation: Application denied due to stable financial profile.'}


In [5]:
# Split into train and validation (90% train / 10% validation)
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)

print(dataset)
print(f"Train size: {len(dataset['train'])}")
print(f"Validation size: {len(dataset['test'])}")

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 900
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 100
    })
})
Train size: 900
Validation size: 100


In [6]:
# Step 2: Train/Validation split
dataset = dataset["train"].train_test_split(test_size=0.1, seed=42)

print(dataset)

print(f"Train size: {len(dataset['train'])}")
print(f"Validation size: {len(dataset['test'])}")

# Peek at a few samples
print("\nSample from train:")
print(dataset["train"][0])

print("\nSample from validation:")
print(dataset["test"][0])

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 810
    })
    test: Dataset({
        features: ['instruction', 'input', 'output'],
        num_rows: 90
    })
})
Train size: 810
Validation size: 90

Sample from train:
{'instruction': 'Explain the credit risk decision for the following applicant profile.', 'input': 'status=no checking account, duration=36, savings=... < 100 DM, employment=1 <= ... < 4 years, amount=3349, age=28', 'output': 'Decision: good. Explanation: Application approved due to stable financial profile.'}

Sample from validation:
{'instruction': 'Explain the credit risk decision for the following applicant profile.', 'input': 'status=... < 100 DM, duration=12, savings=... < 100 DM, employment=1 <= ... < 4 years, amount=1620, age=30', 'output': 'Decision: bad. Explanation: Application denied due to stable financial profile.'}


In [2]:
from huggingface_hub import login
login()

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM

model_name = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(model_name, use_auth_token=True)

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-2-7b-hf.
403 Client Error. (Request ID: Root=1-68d8ee67-024e4a3d649db95b44144e96;39354c8c-0a14-4257-8ddd-7f7fae417bbf)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-2-7b-hf/resolve/main/config.json.
Your request to access model meta-llama/Llama-2-7b-hf is awaiting a review from the repo authors.